# Laboratioan: find optimum

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Experiment
This part defines you experiment. To get data from your experimental setup, just 

```python 
gaussian_function=(x, y, fwhm)
```

which will give you you result 

In [18]:
# Data generation. Doo not touch... This is your physical space :) 

def gaussian_function(x, y, pars=None, add_noise=False):
    # to be changed: generate all data here
    np.random.seed(2)
    if not pars:
        x0 = np.random.randn(1)[0]+0.5
        y0 = x0+0.5
        fwhm = np.random.normal(loc=1.5,scale=3, size=100).mean()
    else:
        x0, y0, fwhm = pars
        
    func = 10*np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)
    
    if add_noise:
        func = add_noise_func(func)
    
    return func, x0, y0, fwhm

def add_noise_func(input_data, mean_noise=0):
    # incomplete: refactor code to generate data + noise in one function
    avg_data = np.mean(input_data)
    noise_ = np.random.normal(mean_noise, np.sqrt(avg_data), 1)[0]
    output_data = input_data + noise_
    
    return output_data

This is how you call for experimental data at x=1 and y=1: 

In [19]:
gaussian_function(1,1)[0]

1.5879227876724087

Use DoE and factioral design to locate the maxima. 

#### Locate maxima in lowest number of trials

In [65]:
inputs_labels = {'x1':'x1',
                 'x2':'x2'}

x1_min = -1
x1_center = 0
x1_max = 1

x2_min = -5
x2_center = 0
x2_max = 5

data_list = [('x1',x1_min,x1_center,x1_max),
        ('x2',x2_min,x2_center,x2_max)]

inputs_df = pd.DataFrame(data_list,columns=['index','low','center','high'])
inputs_df = inputs_df.set_index(['index'])
inputs_df['label'] = inputs_df.index.map( lambda z : inputs_labels[z] )

inputs_df

,low,center,high,label
index,,,,
x1,-1,0,1,x1
x2,-5,0,5,x2


In [66]:
# compute averages and span
inputs_df['average'] = inputs_df.apply( lambda z : ( z['high'] + z['low'])/2 , axis=1)
inputs_df['span'] = inputs_df.apply( lambda z : ( z['high'] - z['low'])/2 , axis=1)

# encode the data
inputs_df['encoded_low'] = inputs_df.apply( lambda z : ( z['low']  - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_center'] = inputs_df.apply( lambda z : ( z['center'] - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_high'] = inputs_df.apply( lambda z : ( z['high'] - z['average'] )/( z['span'] ), axis=1)

inputs_df = inputs_df.drop(['average','span'],axis=1)

inputs_df

,low,center,high,label,encoded_low,encoded_center,encoded_high
index,,,,,,,
x1,-1,0,1,x1,-1.0,0.0,1.0
x2,-5,0,5,x2,-1.0,0.0,1.0


In [67]:
import itertools
encoded_inputs= list(itertools.product([-1,1],[-1,1]))
encoded_inputs
for i in range(0,5):
    encoded_inputs.append((0,0))
encoded_inputs

[(-1, -1), (-1, 1), (1, -1), (1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]

In [68]:
results=pd.DataFrame(encoded_inputs)
results=results[results.columns[::-1]]
results.columns=['x1','x2'] #y will be output
results

,x1,x2
0,-1,-1
1,1,-1
2,-1,1
3,1,1
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0


In [69]:
real_experiment = results
var_labels = []
for var in ['x1','x2']:
    var_label = inputs_df.loc[var]['label']
    var_labels.append(var_label)
    real_experiment[var_label] = results.apply(
        lambda z : inputs_df.loc[var]['low'] if z[var]<0 else (inputs_df.loc[var]['high'] if z[var]>0 else inputs_df.loc[var]['center']), 
        axis=1)

print("The values of each real variable in the experiment:")
real_experiment[var_labels]


The values of each real variable in the experiment:


,x1,x2
0,-1,-5
1,1,-5
2,-1,5
3,1,5
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0


In [70]:
y = []
for i in range(0, len(real_experiment)):
    y.append(gaussian_function(real_experiment['x1'][i],real_experiment['x2'][i])[0])

results['response'] = y
results

,x1,x2,response
0,-1,-5,3.237759e-25
1,1,-5,5.924275e-25
2,-1,5,5.041252e-16
3,1,5,9.224207e-16
4,0,0,5.326879e+00
5,0,0,5.326879e+00
6,0,0,5.326879e+00
7,0,0,5.326879e+00
8,0,0,5.326879e+00


In [72]:
response_values = []
for i in range(len(real_experiment)):
    x1 = real_experiment['x1'][i]
    x2 = real_experiment['x2'][i]
    response = gaussian_function(x1, x2)[0]
    response_values.append(response)

real_experiment['response'] = response_values

In [73]:
x = real_experiment[var_labels].values
y = real_experiment['response'].values

In [84]:
def objective(pars):
    if isinstance(pars, list):
        max_values = []
        for i in range(len(pars)):
            if i == 0:
                func, _, _, _ = gaussian_function(x[:, 0], x[:, 1])
            else:
                func, _, _, _ = gaussian_function(x[:, 0], x[:, 1], pars[i])
            max_values.append(np.max(func))
        return -np.max(max_values)  # Negate the max value for maximization
    else:
        func, _, _, _ = gaussian_function(x[:, 0], x[:, 1], pars)
        return -np.max(func)  # Negate the max value for maximization

In [90]:
def objective(pars):
    max_values = []
    if len(pars) == 0:  # Handle empty pars
        func, _, _, _ = gaussian_function(x[:, 0], x[:, 1])
        max_values.append(np.max(func))
    else:
        for i in range(len(pars)):
            func, _, _, _ = gaussian_function(x[:, 0], x[:, 1], pars[i])
            max_values.append(np.max(func))
    return -np.max(max_values)  # Negate the max value for maximization

In [91]:
from scipy.optimize import minimize
result = minimize(objective, x0=[0, 0, 1], method='Nelder-Mead')

TypeError: cannot unpack non-iterable numpy.float64 object

In [71]:
np.random.randn(1)[0]+0.5

0.8860780476035945

In [11]:
print(np.random.seed(2))

None


In [12]:
np.random.normal(loc=1.5,scale=3, size=100).mean()

1.1887765983522067